In [4]:
import sqlite3
import pandas as pd

# Create an in-memory SQLite database
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()

# Load CSVs into pandas
ipo_df = pd.read_csv("ipos.csv")
fr_df = pd.read_csv("funding_rounds.csv")

# Write the dataframes to the SQLite database
ipo_df.to_sql("ipos", connection, index=False, if_exists="replace")
fr_df.to_sql("funding_rounds", connection, index=False, if_exists="replace")

# Verify the tables were created
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


[('ipos',), ('funding_rounds',)]

In [5]:
# Re-running and formatting SQL Query 1
query1 = """
-- Query 1: Count total number of funding rounds in the dataset
-- This query counts how many funding rounds are recorded in the table
-- It gives a general sense of the scale of available funding data
SELECT COUNT(*) AS total_funding_rounds
FROM funding_rounds;
"""
q1_result = pd.read_sql(query1, connection)
q1_result

,total_funding_rounds
0,52928


In [6]:
# Optimized version of Query 2 using a window function instead of a correlated subquery
query2_optimized = """
-- Query 2 (Optimized): Get the latest funding round amount for IPO companies using a window function
-- This avoids correlated subqueries by ranking funding rounds and selecting only the latest one per company
WITH ranked_funding AS (
    SELECT 
        fr.object_id,
        fr.raised_amount_usd,
        fr.funded_at,
        ROW_NUMBER() OVER (PARTITION BY fr.object_id ORDER BY fr.funded_at DESC) AS row_num
    FROM funding_rounds fr
    JOIN ipos i ON fr.object_id = i.object_id
)
SELECT object_id, raised_amount_usd, funded_at
FROM ranked_funding
WHERE row_num = 1;
"""
q2_optimized_result = pd.read_sql(query2_optimized, connection)
q2_optimized_result

,object_id,raised_amount_usd,funded_at
0,c:100844,3500000.0,2011-09-21
1,c:10241,41000000.0,1997-03-01
2,c:104377,37000000.0,2011-09-12
3,c:10475,100000.0,2011-03-15
4,c:10614,4200000.0,2010-01-19
...,...,...,...
519,c:949,0.0,2012-11-21
520,c:959,4128000.0,2012-12-12
521,c:9636,2732622.0,2001-11-30
522,c:9743,264000000.0,2012-07-09


In [7]:
# SQL Query 3: Join to count number of funding rounds per IPO company
query3 = """
-- Query 3: Join to count number of funding rounds for each IPO company
-- This joins funding_rounds with ipos and groups by object_id to count how many rounds each IPO company had
SELECT fr.object_id, COUNT(*) AS round_count
FROM funding_rounds fr
JOIN ipos i ON fr.object_id = i.object_id
GROUP BY fr.object_id;
"""
q3_result = pd.read_sql(query3, connection)

print(q3_result.shape)  # Confirm number of IPO companies
q3_result.head()        # Preview a few results


(524, 2)


,object_id,round_count
0,c:100844,4
1,c:10241,1
2,c:104377,8
3,c:10475,1
4,c:10614,1


In [8]:
# SQL Query 4: Group by — Average raise by funding round type
query4 = """
-- Query 4: Group average raise by funding round type
-- This shows how funding amounts differ between round types like seed, series A, etc.
SELECT funding_round_type, AVG(raised_amount_usd) AS avg_raised
FROM funding_rounds
GROUP BY funding_round_type;
"""
q4_result = pd.read_sql(query4, connection)
q4_result

,funding_round_type,avg_raised
0,angel,3.056193e+05
1,crowdfunding,1.638457e+06
2,other,1.123907e+07
3,post-ipo,1.694044e+08
4,private-equity,2.502106e+07
5,series-a,5.914058e+06
6,series-b,1.134449e+07
7,series-c+,2.116659e+07
8,venture,8.159983e+06


In [9]:
# SQL Query 5: Window Function — Rank funding rounds per company by size
query5 = """
-- Query 5: Rank funding rounds per company by size using window function
-- This uses RANK() to assign a ranking of funding round size within each company
SELECT 
    object_id, 
    funding_round_id,
    raised_amount_usd,
    RANK() OVER (PARTITION BY object_id ORDER BY raised_amount_usd DESC) AS raise_rank
FROM funding_rounds;
"""
q5_result = pd.read_sql(query5, connection)
q5_result

,object_id,funding_round_id,raised_amount_usd,raise_rank
0,c:1,2312,25000000.0,1
1,c:1,889,9500000.0,2
2,c:1,888,5250000.0,3
3,c:1001,1644,5000000.0,1
4,c:10014,6682,0.0,1
...,...,...,...,...
52923,c:9989,5765,500000.0,2
52924,c:9994,3253,250000.0,1
52925,c:9994,6112,250000.0,1
52926,c:9995,3264,750000.0,1


In [10]:
# SQL Query 6: Window Function — Cumulative funding per company
query6 = """
-- Query 6: Cumulative funding per company using window function
-- This calculates a running total of funds raised per company, ordered by date
SELECT 
    object_id,
    funded_at,
    raised_amount_usd,
    SUM(raised_amount_usd) OVER (
        PARTITION BY object_id 
        ORDER BY funded_at
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS rolling_total
FROM funding_rounds;
"""
q6_result = pd.read_sql(query6, connection)
q6_result

,object_id,funded_at,raised_amount_usd,rolling_total
0,c:1,2005-10-01,5250000.0,5250000.0
1,c:1,2007-01-01,9500000.0,14750000.0
2,c:1,2008-05-19,25000000.0,39750000.0
3,c:1001,2008-02-26,5000000.0,5000000.0
4,c:10014,2008-09-01,0.0,0.0
...,...,...,...,...
52923,c:9989,2009-02-01,500000.0,1100000.0
52924,c:9994,2007-01-01,250000.0,250000.0
52925,c:9994,2008-05-01,250000.0,500000.0
52926,c:9995,2008-08-25,750000.0,750000.0


In [11]:
# SQL Query 7: Window Function — Row number and average raise per company
query7 = """
-- Query 7: Assign row numbers per company and calculate average raise using window function
-- This adds a unique row number per company and shows running average funding amount
SELECT 
    object_id,
    funding_round_id,
    raised_amount_usd,
    ROW_NUMBER() OVER (PARTITION BY object_id ORDER BY funded_at) AS row_num,
    AVG(raised_amount_usd) OVER (PARTITION BY object_id) AS avg_raise
FROM funding_rounds;
"""
q7_result = pd.read_sql(query7, connection)
q7_result

,object_id,funding_round_id,raised_amount_usd,row_num,avg_raise
0,c:1,888,5250000.0,1,13250000.0
1,c:1,889,9500000.0,2,13250000.0
2,c:1,2312,25000000.0,3,13250000.0
3,c:1001,1644,5000000.0,1,5000000.0
4,c:10014,6682,0.0,1,0.0
...,...,...,...,...,...
52923,c:9989,5765,500000.0,2,550000.0
52924,c:9994,3253,250000.0,1,250000.0
52925,c:9994,6112,250000.0,2,250000.0
52926,c:9995,3264,750000.0,1,750000.0


In [12]:
# SQL Query 8: Simulated ROLLUP — Subtotals and totals by round type and year
query8 = """
-- Query 8: Use simulated ROLLUP to get subtotal and total raised amount grouped by round type and year
-- SQLite doesn't support ROLLUP, so we use UNION ALL to simulate it
SELECT funding_round_type, STRFTIME('%Y', funded_at) AS year, SUM(raised_amount_usd) AS total_raised
FROM funding_rounds
GROUP BY funding_round_type, year

UNION ALL

SELECT funding_round_type, NULL AS year, SUM(raised_amount_usd) AS total_raised
FROM funding_rounds
GROUP BY funding_round_type

UNION ALL

SELECT NULL AS funding_round_type, NULL AS year, SUM(raised_amount_usd) AS total_raised
FROM funding_rounds;
"""
q8_result = pd.read_sql(query8, connection)
q8_result

,funding_round_type,year,total_raised
0,angel,None,2.051434e+07
1,angel,1973,0.000000e+00
2,angel,1974,0.000000e+00
3,angel,1984,1.410000e+05
4,angel,1985,0.000000e+00
...,...,...,...
161,series-a,None,5.838949e+10
162,series-b,None,5.549725e+10
163,series-c+,None,8.923834e+10
164,venture,None,1.251905e+11


In [15]:
# SQL Query 9: Subquery — Companies with more than 3 funding rounds
query9 = """
-- Query 9: Subquery to find companies with more than 3 funding rounds
-- The outer query filters based on subquery aggregation
SELECT object_id
FROM funding_rounds
GROUP BY object_id
HAVING COUNT(*) >3;
"""
q9_result = pd.read_sql(query9, connection)
q9_result

,object_id
0,c:10015
1,c:10054
2,c:100844
3,c:1010
4,c:10161
...,...
2371,c:980
2372,c:9836
2373,c:9840
2374,c:9949


In [16]:
# SQL Query 10: Subquery — Funding rounds with raised amount above global average
query10 = """
-- Query 10: Funding rounds with raised amount above global average
-- Subquery calculates the average, outer query filters on it
SELECT *
FROM funding_rounds
WHERE raised_amount_usd > (
    SELECT AVG(raised_amount_usd)
    FROM funding_rounds
);
"""
q10_result = pd.read_sql(query10, connection)
q10_result

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at
0,1,1,c:4,2006-12-01,series-b,b,8500000.0,8500000.0,USD,0.0,...,0.0,None,2,0,0,http://www.marketingvox.com/archives/2006/12/2...,None,initial-importer,2007-07-04 04:52:57,2008-02-27 23:14:29
1,3,3,c:5,2005-05-01,series-a,a,12700000.0,12700000.0,USD,115000000.0,...,0.0,USD,3,0,0,http://www.techcrunch.com/2007/11/02/jim-breye...,Jim Breyer: Extra $500 Million Round For Faceb...,initial-importer,2007-05-27 06:09:10,2013-06-28 20:07:23
2,4,4,c:5,2006-04-01,series-b,b,27500000.0,27500000.0,USD,525000000.0,...,0.0,USD,4,0,0,http://www.facebook.com/press/info.php?factsheet,Facebook Funding,initial-importer,2007-05-27 06:09:36,2013-06-28 20:07:24
3,5,5,c:7299,2006-05-01,series-b,b,10500000.0,10500000.0,USD,0.0,...,0.0,None,2,0,0,http://www.techcrunch.com/2006/05/14/photobuck...,PhotoBucket Closes $10.5M From Trinity Ventures,initial-importer,2007-05-29 11:05:59,2008-04-16 17:09:12
4,7,7,c:9,2007-03-01,series-b,b,10000000.0,10000000.0,USD,100000000.0,...,110000000.0,USD,1,0,0,http://www.techcrunch.com/2007/03/05/100-milli...,100 Million Valuation for Geni,initial-importer,2007-05-31 20:35:03,2009-01-19 18:29:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11903,57922,57922,c:42075,2013-12-12,series-b,b,16000000.0,16000000.0,USD,0.0,...,0.0,USD,1,1,0,http://www.businesswire.com/news/home/20131212...,Frontier Capital Leads Series B Funding to Fue...,ramesh2013,2013-12-12 12:52:19,2013-12-12 12:52:19
11904,57926,57926,c:15847,2013-12-12,venture,unattributed,15000000.0,15000000.0,USD,0.0,...,0.0,USD,4,1,0,http://www.pehub.com/2013/12/mocana-raises-15-...,Mocana raises $15 mln in strategic funding; GE...,ramesh2013,2013-12-12 12:57:35,2013-12-12 12:57:35
11905,57931,57931,c:217789,2013-12-11,venture,unattributed,9329636.0,9329636.0,USD,0.0,...,0.0,USD,0,1,1,http://www.sec.gov/Archives/edgar/data/1589577...,SEC,amit2013,2013-12-12 13:10:24,2013-12-12 13:10:24
11906,57934,57934,c:286184,2013-12-11,venture,unattributed,12999000.0,12999000.0,USD,0.0,...,0.0,USD,0,1,1,http://www.sec.gov/Archives/edgar/data/1552224...,SEC,amit2013,2013-12-12 13:17:16,2013-12-12 13:17:16


In [17]:
# SQL Query 11: CTE — Calculate average funding per company, then filter those with avg > $10M
query11 = """
-- Query 11: Use a CTE to calculate average funding per company
-- Then select only those companies whose average raise exceeds $10M
WITH avg_funding AS (
    SELECT object_id, AVG(raised_amount_usd) AS avg_raised
    FROM funding_rounds
    GROUP BY object_id
)
SELECT *
FROM avg_funding
WHERE avg_raised > 10000000;
"""
q11_result = pd.read_sql(query11, connection)
q11_result

,object_id,avg_raised
0,c:1,1.325000e+07
1,c:10015,1.361384e+07
2,c:10018,1.100000e+07
3,c:1005,2.000000e+07
4,c:10054,1.735714e+07
...,...,...
4606,c:9891,1.080000e+08
4607,c:98929,2.537820e+07
4608,c:9949,7.162500e+07
4609,c:997,1.450000e+07


In [18]:
# Re-run SQL Query 12 (simulated stored procedure via function) for IPOs in a specific year
query12 = """
-- Query 12: Simulated stored procedure using a parameterized query
-- Returns IPOs from a given year and the raised amount in their last funding round
WITH latest_rounds AS (
    SELECT 
        fr.object_id,
        fr.raised_amount_usd,
        fr.funded_at,
        ROW_NUMBER() OVER (PARTITION BY fr.object_id ORDER BY fr.funded_at DESC) AS rn
    FROM funding_rounds fr
    JOIN ipos i ON fr.object_id = i.object_id
    WHERE STRFTIME('%Y', i.public_at) = '2013'
)
SELECT object_id, raised_amount_usd, funded_at
FROM latest_rounds
WHERE rn = 1;
"""
q12_result = pd.read_sql(query12, connection)
q12_result

,object_id,raised_amount_usd,funded_at
0,c:104377,37000000.0,2011-09-12
1,c:1063,10000000.0,2012-05-23
2,c:10704,2400000.0,2012-04-01
3,c:10795,0.0,2013-11-25
4,c:1105,40000000.0,2011-03-28
...,...,...,...
76,c:72361,6750000.0,2013-02-28
77,c:73152,10000000.0,2012-04-20
78,c:75365,25000000.0,2013-07-18
79,c:81355,150000.0,2013-11-14


In [20]:
# Query 13: All funding rounds ordered from largest to smallest
query13 = """
-- Query 13: All funding rounds ordered by amount raised (largest to smallest)
-- This helps identify the biggest funding rounds in the dataset
SELECT *
FROM funding_rounds
ORDER BY raised_amount_usd DESC;
"""
q13_result = pd.read_sql(query13, connection)
q13_result

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at
0,47231,47231,c:4843,2012-03-05,other,debt_round,3.835050e+09,3.835050e+09,USD,0.0,...,0.0,USD,0,1,0,http://www.sec.gov/Archives/edgar/data/732712/...,SEC,bapi2013,2013-08-30 12:06:45,2013-08-30 12:06:45
1,40117,40117,c:216492,2013-03-01,other,debt_round,3.822518e+09,3.822518e+09,USD,0.0,...,0.0,USD,0,1,1,http://www.sec.gov/Archives/edgar/data/732712/...,SEC,santosh2013,2013-06-18 09:57:39,2013-06-18 10:20:07
2,10413,10413,c:13219,2008-05-06,post-ipo,post_ipo_equity,3.200000e+09,3.200000e+09,USD,0.0,...,0.0,None,5,0,1,http://www.techcrunch.com/2008/05/06/32-billio...,$3.2 Billion WiMax Deal Goes Through. Take Cover.,sophia,2009-12-18 05:21:49,2012-02-24 19:10:07
3,44097,44097,c:242735,2013-07-29,private-equity,private_equity,2.600000e+09,2.600000e+09,USD,0.0,...,0.0,USD,1,1,1,http://www.finsmes.com/2013/07/sigmacare-secur...,SigmaCare Secures Equity Funding,prasant2013,2013-07-30 04:33:10,2013-07-31 04:34:14
4,39835,39835,c:64365,2013-06-12,other,debt_round,2.400000e+09,2.400000e+09,USD,0.0,...,0.0,USD,0,1,1,http://www.pehub.com/2013/06/12/onex-backed-ca...,Onex-Backed Carestream Raises Debt for $725M D...,prasant2013,2013-06-13 09:32:21,2013-06-13 09:32:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,57880,57880,c:285244,2011-01-01,series-a,a,0.000000e+00,1.615000e+06,EUR,0.0,...,0.0,USD,3,0,0,None,None,sandernijssen,2013-12-12 10:38:45,2013-12-12 12:51:11
52924,57882,57882,c:285244,2009-01-01,angel,angel,0.000000e+00,3.500000e+05,EUR,0.0,...,0.0,USD,3,0,1,None,None,sandernijssen,2013-12-12 10:41:27,2013-12-12 12:51:16
52925,57912,57912,c:60389,2013-07-10,series-c+,c,0.000000e+00,0.000000e+00,EUR,0.0,...,0.0,USD,1,1,1,http://www.rockethome.de/news/press/growth-cap...,Growth Capital for ROCKETHOME,crunchrockethome,2013-12-12 12:32:53,2013-12-12 13:02:54
52926,57940,57940,c:65946,2013-12-12,venture,unattributed,0.000000e+00,0.000000e+00,USD,0.0,...,0.0,USD,0,1,0,http://finance.yahoo.com/news/avinger-complete...,Avinger Completes Major New Financing,amit2013,2013-12-12 13:33:40,2013-12-12 13:33:40


In [22]:
# Query 14: Count distinct investors across all rounds
query14 = """
-- Query 14: Count total number of distinct participants across all funding rounds
-- This gives a sense of the diversity of the investor base
SELECT COUNT(DISTINCT participants) AS unique_participant_counts
FROM funding_rounds;
"""
q14_result = pd.read_sql(query14, connection)
q14_result

,unique_participant_counts
0,32


In [25]:
# Query 15: Average time between funding rounds for companies with 2+ rounds
query15 = """
-- Query 15: Average days between rounds per company (requires valid dates)
-- This helps evaluate funding cadence for more active startups
WITH dated_rounds AS (
    SELECT object_id, funded_at,
           JULIANDAY(funded_at) AS date_jul
    FROM funding_rounds
    WHERE funded_at IS NOT NULL
),
round_diffs AS (
    SELECT 
        object_id,
        date_jul - LAG(date_jul) OVER (PARTITION BY object_id ORDER BY date_jul) AS days_between
    FROM dated_rounds
)
SELECT AVG(days_between) AS avg_days_between_rounds
FROM round_diffs
WHERE days_between IS NOT NULL;
"""
q15_result = pd.read_sql(query15, connection)
q15_result

,avg_days_between_rounds
0,460.407926
